# The Online Cafeteria
## Final Exam in Programming, Algorithms and Data Structures
### Daniel Henke, 176182

#### Customer Class

In [314]:
# to be treated as an abstract class
class General_Customer:
    def __init__(self, name, customer_id, university, customer_type, discount=0):
        self.name = name
        self.customer_id = customer_id
        self.university = university
        self.customer_type = customer_type
        self.discount = discount
        self.balance = 0
        self.orders = []
    
    def str(self):
        if self.customer_id == None:
            return f"{self.name} ({self.customer_type})"   
        else:
            return f"{self.name} - {self.customer_id} ({self.customer_type})"    
    
    def view_menu(self, cafeteria_name):
        """Lets the customer view the menu of a cafeteria and the prices of the items after applying the discount.

        Args:
            cafeteria_name (String): the cafeteria name whose menu the customer wants to view
        """
        cafeteria=self.university.get_cafeteria(cafeteria_name)
        assert cafeteria!=None, f"Sorry, {cafeteria_name} is not available in the university"
        print(f"Menu for {cafeteria.name}:")
        for item, details in cafeteria.menu.items():
            print(f"{item}: {details['price']*(1-(self.discount//100))}dkk ({details['quantity']} available)")
    
    def view_detailed_menu(self, cafeteria_name):
        """Lets the customer view the detailed menu of a cafeteria.

        Args:
            cafeteria_name (String): the cafeteria name whose menu the customer wants to view
        """
        cafeteria=self.university.get_cafeteria(cafeteria_name)
        assert cafeteria!=None, f"Sorry, {cafeteria_name} is not available in the university"
        print(f"Detailed Menu for {cafeteria.name}:")
        for item, details in cafeteria.menu.items():
            print(f"{item}: {details['description']}")
            print(f"{details['price']}dkk ({details['quantity']} available)")
    
    def add_balance(self, amount):
        """Adds balance to the account.

        Args:
            amount (int): the amount to be added to the balance
        """
        assert amount>0, "Amount should be greater than 0"
        self.balance+=amount
        print(f"${amount} added to the balance of {self.name}")
         
    def place_order(self, cafeteria_name, item, quantity):
        """Places an order in a cafeteria.

        Args:
            cafeteria_name (String): the cafeteria where the order is to be placed
            item (string): the item to be ordered
            quantity (int): number of items to be ordered
        """
        cafeteria=self.university.get_cafeteria(cafeteria_name)
        assert cafeteria!=None, f"Sorry, {cafeteria_name} is not available in the university"
        assert quantity>0, "Quantity should be greater than 0"
        if item in cafeteria.menu:
            #calculating the price after discount for full order           
            price = cafeteria.menu[item]['price']*quantity*(1-(self.discount//100))
            if self.balance < price:
                raise ValueError(f"Sorry, you do not have enough balance to place this order")
            
            #placing the order with the cafeteria
            order=cafeteria.process_order(self.customer_id, self.customer_type, item, quantity, self.discount)
            
            #updating the balance based on the actual order (may be lower quantity)
            self.balance-=order.price
            self.orders.append(order)
            print(f"Order placed by {self.name} for {order.quantity} {item}(s) for {order.price}dkk")
                
        else:
            raise ValueError(f"Sorry, {item} is not available in the menu")
        
    
    def view_orders(self):
        """Lets the customer view all the orders placed by them and not picked up yet."""
        print(f"Orders placed by {self.name}:")
        for order in self.orders:
            print(order)
    
    def get_balance(self):
        """Returns the balance of the customer."""
        return self.balance
    
    def pick_up_order(self, order_id):
        """Lets the customer pick up an order from a cafeteria.

        Args:
            order_id (int): the id of the order to be picked up
        """
        for order in self.orders:
            if order.order_id == order_id:
                order.pick_up()
                self.orders.remove(order)
                return
        raise ValueError(f"Sorry, order with id {order_id} not found")
        

#### Student
The first subclass of customer

In [315]:
class Student(General_Customer):
    def __init__(self, name, student_id, university):
        super().__init__(name, student_id, university, "Student", 20)

#### Staff

In [316]:
class Staff(General_Customer):
    def __init__(self, name, staff_id, university):
        super().__init__(name, staff_id, university, "Staff", 10)

#### Guest
Guests cannot order online and have no account. They can merely view the menu.

In [317]:
class Guest(General_Customer):
    def __init__(self, name):
        super().__init__(name, None, "Guest", 0)
    
    def place_order(self, cafeteria, item, quantity):
        raise PermissionError("Guests cannot place orders online. Please book in person.")
    
    def add_balance(self, amount):
        raise PermissionError("Guests do not have a balance to add to.")
    

#### Cafeteria

In [318]:
import copy

class Cafeteria:
    def __init__(self, name, university):
        self.name = name
        self.university = university
        self.menu = {}
        self.orders = []
        self.item_popularity = {}
        self.revenue=0
        
    def add_item(self, item, description, price, quantity):
        """Adds an item to the menu of the cafeteria.

        Args:
            item (string): the name of the item
            description (string): the description of the item
            price (int): the price of the item
            quantity (int): the quantity of the item available
        """
        assert price>0, "Price should be greater than 0"
        assert quantity>0, "Quantity should be greater than 0"
        self.menu[item] = {'description': description, 'price': price, 'quantity': quantity, 'cafeteria': self.name}
        self.university.update_sorted_menu(item, quantity, self.name, description, price)
        
    
    def upload_menu(self, new_menu):
        """Uploads the menu to the cafeteria.

        Args:
            menu (dict): a dictionary containing the items and their details
        """
        assert type(new_menu)==dict, "Menu should be a dictionary"
        self.menu = copy.deepcopy(new_menu)
        for key in self.menu.keys():
            self.menu[key]['cafeteria'] = self.name
        self.university.is_sorted = False
        
    def update_item(self, item, description, price, quantity):
        """Updates the price and quantity of an item in the menu
        Args:
            item (string): the name of the item
            description (string): the new description of the item
            price (int): the new price of the item
            quantity (int): the new quantity of the item
        """
        assert price>0, "Price should be greater than 0"
        assert quantity>0, "Quantity should be greater than 0"
        if item in self.menu:
            self.menu[item]['description'] = description
            self.menu[item]['price'] = price
            self.menu[item]['quantity'] = quantity
            self.university.is_sorted = False
        else:
            raise ValueError(f"Sorry, {item} is not available in the menu")
        
    def restock_item(self, item, quantity):
        """Restocks an item in the menu.

        Args:
            item (string): the name of the item
            quantity (int): the quantity to be added to the stock
        """
        assert quantity>0, "Quantity should be greater than 0"
        if item in self.menu:
            self.menu[item]['quantity']+=quantity
            self.university.update_sorted_menu(item, self.menu[item]['quantity'], self.name, self.menu[item]['description'], self.menu[item]['price'])
        else:
            raise ValueError(f"Sorry, {item} is not available in the menu")
        
    def remove_item(self, item):
        """Removes an item from the menu.

        Args:
            item (string): the name of the item
        """
        if item in self.menu:
            self.menu.pop(item)
        else:
            raise ValueError(f"Sorry, {item} is not available in the menu")
    
    def process_order(self, customer_id, customer_type, item, quantity, discount=0):
        """Processes an order placed by a customer.

        Args:
            customer_id (int): the id of the customer placing the order
            customer_type (string): the type of the customer placing the order
            item (string): the item to be ordered
            quantity (int): number of items to be ordered
            discount (int): the discount to be applied to the order

        Returns:
            object: the order object
        """
        assert quantity>0, "Quantity should be greater than 0"
        assert discount>=0, "Discount should be greater than or equal to 0"
        assert discount<=100, "Discount should be less than or equal to 100"
        assert self.university.get_customer(customer_id)!=None, f"Sorry, customer with id {customer_id} not found"
        if item in self.menu:
            if self.menu[item]['quantity'] < quantity and self.menu[item]['quantity'] > 0:
                print(f"Sorry, only {self.menu[item]['quantity']} {item}(s) available")
                quantity = self.menu[item]['quantity']
            elif self.menu[item]['quantity'] == 0:
                raise ValueError(f"Sorry, {item} is out of stock")
            
            order = Order(self, customer_id, customer_type, item, quantity, self.menu[item]['price'], discount)
            if item not in self.item_popularity:
                self.item_popularity[item] = 0
            self.item_popularity[item] += quantity
            self.orders.append(order)
            self.print_orders()
            return order
        else:
            raise ValueError(f"Sorry, {item} is not available in the menu")
        
    def print_orders(self):
        """Prints all the orders placed in the cafeteria."""
        print(f"Orders in {self.name}:")
        for order in self.orders:
            print(order)
            
    
    def complete_order(self, order_id):
        """Completes an order placed by a customer.

        Args:
            order (int): the order object to be completed
        """
        for order in self.orders:
            if order.order_id == order_id:
                self.orders.remove(order)
                self.menu[order.item]['quantity']-=order.quantity
                self.university.update_sorted_menu(order.item, order.quantity, self.name)
                self.revenue+=order.price
                order.complete()
                return
        raise ValueError(f"Order {order_id} not found")
        
    
    def cancel_order(self, order_id):
        """Cancels an order placed by a customer.

        Args:
            order (int): the order object to be cancelled
        """
        for order in self.orders:
            if order.order_id == order_id:
                self.orders.remove(order)
                order.cancel()
        raise ValueError(f"Order {order_id} not found")
    
    def close_cafeteria(self):
        """Closes the cafeteria for the day and prints the revenue generated."""
        print(f"Revenue generated today by {self.name}: {self.revenue}dkk")
        for order in self.orders:
            order.cancel()
        self.item_popularity = {}
        self.menu = {}
        self.university.is_sorted = False
        self.revenue = 0
    
    def popular_items(self, n):
        """Prints the n most popular items in the cafeteria.

        Args:
            n (int): the number of items to be printed
        """
        assert n>0, "n should be greater than 0"
        def merge_sort(items):
            if len(items) <= 1:
                return items

            # Split into two halves
            mid = len(items) // 2
            left_half = merge_sort(items[:mid])
            right_half = merge_sort(items[mid:])

            # Merge sorted halves
            return merge(left_half, right_half)

        def merge(left, right):
            sorted_items = []
            while len(left)>0 and len(right)>0:
                # Compare the second element (popularity)
                if left[0][1] >= right[0][1]:
                    sorted_items.append(left.pop(0))
                else:
                    sorted_items.append(right.pop(0))

            # Append any remaining items
            if len(left) > 0:
                sorted_items.extend(left)
            elif len(right) > 0:
                sorted_items.extend(right)
            return sorted_items
                
        
        # Convert dictionary to a list of tuples (item, popularity)
        items = list(self.item_popularity.items())
        # Sort using merge sort
        sorted_items = merge_sort(items)
        # Return the top N items
        return sorted_items[:n]

#### Order

In [319]:
class Order:
    class_counter=1
    def __init__(self, cafeteria, customer_id, customer_type, item, quantity, price, discount=0):
        self.order_id = Order.class_counter
        self.cafeteria = cafeteria
        self.customer_id = customer_id
        self.customer_type = customer_type
        self.item = item
        self.quantity = quantity
        self.price = price*quantity*(1-(discount//100))
        #for simplicity, we assume all orders are accepted as the check is done before creating the order
        self.status = "Accepted"
        Order.class_counter+=1
        
    def __str__(self):
        return f"({self.status}) Order {self.order_id} by {self.customer_type} {self.customer_id} for {self.quantity} {self.item}(s) for {self.price}dkk"	
    
    def total_price(self):
        return self.price
    
    def complete(self):
        self.status = "Completed"
        print(f"Order {self.order_id} completed")
    
    def cancel(self):
        self.status = "Cancelled"
        #Refund
        for customer in self.cafeteria.university.all_customers():
            if customer.customer_id == self.customer_id:
                customer.balance+=self.price
        print(f"Order {self.order_id} cancelled")
        
        
    def pick_up(self):
        self.status = "Picked Up"
        print(f"Order {self.order_id} picked up")
        
    

#### University


In [320]:
class University:
    def __init__(self, name):
        self.name = name
        self.cafeterias = []
        self.students = []
        self.staff = []
        self.sorted_menu = []
        self.is_sorted = False
        
    def add_student(self, name, student_id):
        assert student_id not in [student.customer_id for student in self.students], f"Student with id {student_id} already exists"
        student = Student(name, student_id, self)
        self.students.append(student)
        return student
    
    def add_staff(self, name, staff_id):
        assert staff_id not in [staff.customer_id for staff in self.staff], f"Staff with id {staff_id} already exists"
        staff = Staff(name, staff_id, self)
        self.staff.append(staff)
        return staff
    
    def all_customers(self):
        return self.students + self.staff
    
    def get_customer(self, customer_id):
        for customer in self.all_customers():
            if customer.customer_id == customer_id:
                return customer
        return None
    
    def add_cafeteria(self, name):
        assert name not in [cafeteria.name for cafeteria in self.cafeterias], f"Cafeteria with name {name} already exists"
        cafeteria = Cafeteria(name, self)
        self.cafeterias.append(cafeteria)
        return cafeteria
    
    def get_cafeteria(self, name):
        for cafeteria in self.cafeterias:
            if cafeteria.name == name:
                return cafeteria
        return None
    
    def generate_customers(self, n_students, n_staff):
        for i in range(n_students):
            self.add_student(f"Student {i+1}", i+1)
        for i in range(n_staff):
            self.add_staff(f"Staff {i+1}", i+1)
    
    def update_sorted_menu(self, item, quantity, cafeteria_name, description=None, price=None):
        """Updates the sorted menu of the university.

        Args:
            item (string): the name of the item
            quantity (int): the quantity of the item available
            cafeteria_name (string): the name of the cafeteria
            description (string): the description of the item
            price (int): the price of the item
        """
        assert quantity>0, "Quantity should be greater than 0"
        assert cafeteria_name in [cafeteria.name for cafeteria in self.cafeterias], f"Sorry, {cafeteria_name} is not available in the university"
        if self.is_sorted:
            for i in range(len(self.sorted_menu)):
                if self.sorted_menu[i][0] == item:
                    # Remove the item from the list
                    prev_item = self.sorted_menu.pop(i)

                    # Re-insert the item at the correct position
                    j = len(self.sorted_menu) - 1
                    while j >= 0 and self.sorted_menu[j][0] > item[0]:
                        j -= 1
                    if description == None:
                        self.sorted_menu.insert(j + 1, (item, prev_item[1], prev_item[2], quantity, cafeteria_name))
                    else:
                        self.sorted_menu.insert(j + 1, (item,  description, price, quantity, cafeteria_name))
                    break
        
    def view_sorted_menu(self):
        """Prints the sorted menu of the university."""
        if not self.is_sorted:
            self.sort_menu()
        print(f"Sorted Menu for {self.name}:")
        for item in self.sorted_menu:
            print(f"{item[0]}:")
            print(f"  {item[4]}: {item[1]} - {item[2]}dkk ({item[3]} available)")
    
    def sort_menu(self):
        """
        Generates a sorted complete menu using insertion sort and updates the cache.
        :return: The sorted complete menu.
        """
        complete_menu = []        
        i=0
        # Sort using insertion sort
        for cafeteria in self.cafeterias:
   
            for name in cafeteria.menu.keys():
                item=(name, cafeteria.menu[name]['description'], cafeteria.menu[name]['price'],cafeteria.menu[name]['quantity'],cafeteria.menu[name]['cafeteria'])
                complete_menu.append(item)
                j=i-1
                while j >= 0 and (complete_menu[j][0] > name or (complete_menu[j][0] == name and complete_menu[j][4]>item[4])):  # Sort by item name
                    complete_menu[j + 1] = complete_menu[j]
                    j -= 1
                complete_menu[j + 1] = item
                i+=1

        # Update the cache
        self.sorted_menu = complete_menu
        self.is_sorted = True
        return self.sorted_menu
    
    def search_menu(self, item_name):
        """Searches for an item in the menu of the university.

        Args:
            item_name (String): the name of the item to be searched

        Returns:
            list: a list of tuples containing the item details
        """
        # Sort the menu if it is not up-to-date
        if not self.is_sorted:
            self.sort_menu()

        # Perform binary search
        low, high = 0, len(self.sorted_menu) - 1
        results = []

        while low <= high:
            mid = (low + high) // 2
            mid_item = self.sorted_menu[mid][0]

            if mid_item == item_name:
                # Find all matches
                results.append(self.sorted_menu[mid])

                # Check neighbors for duplicates
                left, right = mid - 1, mid + 1
                while left >= 0 and self.sorted_menu[left][0] == item_name:
                    results.append(self.sorted_menu[left])
                    left -= 1
                while right < len(self.sorted_menu) and self.sorted_menu[right][0] == item_name:
                    results.append(self.sorted_menu[right])
                    right += 1
                return results

            elif mid_item < item_name:
                low = mid + 1
            else:
                high = mid - 1

        return f"Item '{item_name}' not found in any cafeteria."
    
    def close_university(self):
        """Closes the university for the day and prints the revenue generated."""
        total_revenue = 0
        for cafeteria in self.cafeterias:
            total_revenue += cafeteria.revenue
            cafeteria.close_cafeteria()
        print(f"Total revenue generated today by {self.name}: {total_revenue}dkk")
        
    

#### Test

In [321]:
#creating a university
university = University("CBS")

#adding a cafeteria to the university
cafeteria1 = university.add_cafeteria("Solbjerg Plads")
cafeteria2 = university.add_cafeteria("Dalgas Have")
cafeteria3 = university.add_cafeteria("Kilen")
cafeteria4 = university.add_cafeteria("Porcelænshaven")

#adding items to the menu of the cafeterias
daily_menu = { "Coffee": {"description": "A standard filter.", "price": 25, "quantity": 10},
         "Tea": {"description": "A cup of green or black tea", "price": 20, "quantity": 15},
         "Sandwich": {"description": "A chicken or tomato morzarella sandwich", "price": 35, "quantity": 5},
         "Kannelbullar": {"description": "A Danish cinnamon bun", "price": 15, "quantity": 20},
         "Croissant": {"description": "A buttery croissant", "price": 20, "quantity": 10},
            "Salad": {"description": "A fresh salad with vegetables", "price": 40, "quantity": 5}}

lunch_menu = { "Pasta": {"description": "Pasta with tomato sauce", "price": 45, "quantity": 10},
            "Pizza": {"description": "A slice of pizza", "price": 30, "quantity": 15},
            "Burger": {"description": "A beef or veggie burger", "price": 50, "quantity": 5},
            "Sushi": {"description": "A sushi roll", "price": 35, "quantity": 20},
            "Soup": {"description": "A bowl of soup", "price": 25, "quantity": 10}}

reduced_menu = { "Pasta": {"description": "Pasta with tomato sauce", "price": 45, "quantity": 10},
            "Pizza": {"description": "A slice of pizza", "price": 30, "quantity": 5},
            "Burger": {"description": "A beef or veggie burger", "price": 50, "quantity": 5}}

drink_menu = { "Coca Cola": {"description": "A can of Coca Cola", "price": 15, "quantity": 10},
            "Fanta": {"description": "A can of Fanta", "price": 15, "quantity": 10},
            "Faxe Kondi": {"description": "A can of Faxe Kondi", "price": 15, "quantity": 10},
            "Water": {"description": "A bottle of water", "price": 10, "quantity": 10}}

snack_menu = { "Chips": {"description": "A bag of chips", "price": 10, "quantity": 10},
            "Chocolate": {"description": "A chocolate bar", "price": 10, "quantity": 10},
            "Gum": {"description": "A pack of gum", "price": 5, "quantity": 10},
            "Popcorn": {"description": "A bag of popcorn", "price": 15, "quantity": 10}}


university.generate_customers(1000,100)

cafeteria1.upload_menu(daily_menu|snack_menu|lunch_menu)
cafeteria2.upload_menu(daily_menu|lunch_menu)
cafeteria3.upload_menu(daily_menu|reduced_menu)
cafeteria4.upload_menu(daily_menu|drink_menu)


Student1 = university.students[0]

try:
   Student1.place_order("Kilen", "Coffee", 1)
except Exception as e:
    print(e)

Student1.add_balance(1000)

Student1.place_order("Solbjerg Plads", "Coffee", 2)

cafeteria1.complete_order(1)

Student1.view_orders()

Student1.pick_up_order(1)

Student1.get_balance()

Student1.place_order("Solbjerg Plads", "Burger", 6)

print(cafeteria1.popular_items(3))

cafeteria1.restock_item("Burger", 5)

Student1.view_menu("Solbjerg Plads")

university.close_university()

Student1.get_balance()

#university.view_sorted_menu()

#university.search_menu("Burger")


Sorry, you do not have enough balance to place this order
$1000 added to the balance of Student 1
Orders in Solbjerg Plads:
(Accepted) Order 1 by Student 1 for 2 Coffee(s) for 50dkk
Order placed by Student 1 for 2 Coffee(s) for 50dkk
Order 1 completed
Orders placed by Student 1:
(Completed) Order 1 by Student 1 for 2 Coffee(s) for 50dkk
Order 1 picked up
Sorry, only 5 Burger(s) available
Orders in Solbjerg Plads:
(Accepted) Order 2 by Student 1 for 5 Burger(s) for 250dkk
Order placed by Student 1 for 5 Burger(s) for 250dkk
[('Burger', 5), ('Coffee', 2)]
Menu for Solbjerg Plads:
Coffee: 25dkk (8 available)
Tea: 20dkk (15 available)
Sandwich: 35dkk (5 available)
Kannelbullar: 15dkk (20 available)
Croissant: 20dkk (10 available)
Salad: 40dkk (5 available)
Chips: 10dkk (10 available)
Chocolate: 10dkk (10 available)
Gum: 5dkk (10 available)
Popcorn: 15dkk (10 available)
Pasta: 45dkk (10 available)
Pizza: 30dkk (15 available)
Burger: 50dkk (10 available)
Sushi: 35dkk (20 available)
Soup: 25d

950

#### Visual Interface